<a href="https://colab.research.google.com/github/Ahtesham519/Praticle_nlp_book/blob/main/BioBert_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests==2.23.0
!pip install pytorch-transformers==1.2.0
!pip install transformers==4.7.0
!pip install pandas==1.1.5
!pip install pytorch-pretrained-bert==0.6.2
!pip install pytorch-nlp==0.5.0
!pip install tensorflow==1.14.0
!pip install torch==1.9.0
!pip install scikit-learn==0.21.3
!pip install tqdm==4.41.1
!pip install matplotlib==3.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      S

In [2]:
# To install the requirements for the entire chapter, uncomment the lines below and run this cell

try:
  import google.colab
    !curl  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch10/ch10-requirements.txt | xargs -n 1 -L 1 pip install
except ModuleNotFoundError:



IndentationError: ignored

In [ ]:
import requests
import tarfile 
import os 
import shutil
import warnings
warnings.filterwarnings("ignore", category = FutureWarnongs)

In [ ]:
def download_file_from_google_drive(id, destination):
  URL = "https://docs.google.com/uc?export=download"

  session = requests.Session()

  response = session.get(URL, params = {'id' :id}, stream = True)
  token = get_confirm_token(response)

  if token:
    params = { 'id' : id, 'confirm' :token}
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

def get_confirm_token(response):
  for key, value in response.cookies.items():
    if key.startwith('download_warning'):
      return value
  
  return None

def save_response_content(response, destination):
  CHUNK_SIZE = 32768

  with open(destination, "wb") as f:
    for chunk in response.iter_content(CHUNK_SIZE):
      if chunk:
        f.write(chunk)

        

In [ ]:
try:
  import google.colab
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget  --quiet  --save-cookies /tmp/cookies)"

except ModuleNotFoundError:
  file_id = '**************'
  destination = 'Data/biobert_weights'
  download_file_google_drive(file_id, destination)

In [3]:
try:
  import google.colab
  !tar -xzf biobert_weights
  !ls biobert_v1.1_pubmed/

except ModuleNotFoundError:

  tar = tarfile.open('Data/biobert_weights', 'r')
  tar.extractall('Data/')
  tar.close()

  print(os.listdir('Data/biobert_v1.1_pebmed'))

  

tar (child): biobert_weights: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
ls: cannot access 'biobert_v1.1_pubmed/': No such file or directory


In [4]:
PATH = "."
try:
  import google.colab
  PATH = './'
except ModuleNotFoundError:
  PATH = 'Data'

In [5]:
try:
  import google.colab
  !transforms-cli convert  --model_type bert --tf_checkpoint  biobert_v1.1_pubmed/model.ckpt-1000000  --config biobert_v1.1

except ModuleNotFoundError:
    !transforms-cli convert  --model_type bert --tf_checkpoint  biobert_v1.1_pubmed/model.ckpt-1000000  --config biobert_v1.1


/bin/bash: transforms-cli: command not found


In [ ]:
df.shape

In [ ]:
df['medical_specialty'].value_counts()

In [ ]:
DATA_DIR = "."
import os
import numpy as np
import pickle
import tensorflow as tf

#Bert imports

import torch
from torch.utils.data import TensorDataset, DataLoader , RandomSampler , SequentialSampler
from tensorflow.keras.perprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BerttTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [9]:
#specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


if torch.cuda.is_available():
  n_gpu = torch.cuda.device_count()
  torch.cuda.get_device_name(0)

In [ ]:
from sklearn.preprocessing import LabelEncoder
#sentiment is positive and negative we  need to convert it to 0 and 1
le = LabelEncoder()
df["medical_specialty"] = le.fit_transform(df["medical_specialty"])
df.head()

In [ ]:
description = list(df['description'])
#Tokenize with bert tokenizer
tokenizer = BertTokenizer.from_pretrained(PATH +'biobert_v1.1_pubmed' , do_lower_case = True)


#Restricting the max size of tokens to 512(BERT doesent accept any more than this)
tokenized_texts = list(map(lambda t: ['[CLS]'] +tokenizer.tokenize(t) + ['[SEP]'] ,description))
print(tokenized_texts[0])

In [ ]:
classes = list(df['medical_specialty'])

In [ ]:
#SET the maximum sequence length
MAX_LEN = 128

#pad our input tokens so that everthing has a uniform length
input_ids = pad_sequences(list(map(tokenizer.conert_tokens_to_ids, tokenized_texts)), 
                          maxlen = MAX_LEN , dtype = "long", truncating= "post" , padding ="post")

In [ ]:
#Use bert tokenizer to convert the tokens to their index numbers in the bert vicabulary
input_ids = [tokenizer.converet_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype ="long" , truncating = "post", padding= "post")

In [ ]:
#Create attention masks 
attention_masks = []
#Create a mask 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
batch_size = 16

#Use train_test_split to split our data into train and validation sets for training 
train_inputs, validation_inputs, train_labels , validation_labels = train_test_split(input_ids ,classes,
                                                                                     random_state = 2020 , test_size = 0.1)
train_masks, validation_masks, _ , _ = train_test_split(attention_masks , input_ids,
                                                        random_state = 2020, test_size = 0.1)

#convert all of our data into torch tensors, the required datatypes for our model
trin_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

#Create an iterator of our data with torch DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler= train_sampler ,batch_size = batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader - DataLoader(validation_Data, sampler = validation_sampler, batch_size = batch_size)



In [ ]:
#Loading pre-trained BERT
model = BertForSequenceClassification.from_pretrained(PATH +'biobert_v1.1_pubmed, num_labels = 40') #binary classification

if torch.cuda.is_available():
  print(model.cuda())
else:
  print(model)
  

In [ ]:
#BERT Tuning 
params_optimizer = list(model.named_parameters())
no_decay = ['bias' , 'gamma' , 'beta']
optimizer_grouped_parameters =[
    {'params' :[p for n,p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
     {'params': [p for n ,p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr = 2e-5,
                     warmup = .1)

#Function to calculate the accuracy of our predictions vs labels

def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis =1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)


torch.cuda.empty_cache()
#Store our loss and accurecy for ploting
train_loss_set = []
#number of training epochs
epochs = 4

#Bert training loop
for _ in trange(epochs, desc='Epoch'):
  ##Training
  model.train()
  #Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_steps = 0, 0
  #train the data for one epochs
  for step, batch in enumerate(train_dataloader):
    #Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    #Unpack the inputs from the dataloader
    b_input_ids , b_input_mask, b_labels = batch
    #Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    #Forward_pass
    loss = model(b_input_ids , token_type_ids = None , attention_mask= b_input_mask, labels= b_labels)
    train_loss_set.append(loss.item())
    #Backward Pass
    loss.backward()
    #Update parameterrs and take a step using the computed fradients
    optimizer.step()
    #Update tracking variables 
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))

##Validation 

#Put model in evulation mode


In [ ]:
#Put model in evulatation mode
model.eval()
#Tracking variables
eval_loss, eval_accuracy = 0,0
nb_eval_steps , nb_eval_examples = 0,0
#Evulate data for one epochs
for batch in validation_dataloader:
  #Add batch to GPU 
  batch = tuple(t.to(device) for t in batch)
  #Unpack the inputs from our dataloader
  b_inputs_ids, b_input_masks, b_labels = batch
  #Telling the model not to compute or store graidents, saving memory and speeding up validation 
  with torch.nograd():
    #Forward pass, calculate logit_prediction
    logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
  #Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  tmp_eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1
print("Validation Accuracy:{}" .format(eval_accuracy/nb_eval_steps))



#Plot training performance
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

